# Q-Learning 深度教程

---

## 学习目标

通过本教程，你将：

1. **深入理解** 时序差分学习 (TD Learning) 的核心思想
2. **掌握** Q-Learning 算法的数学原理与实现细节
3. **实现** 从零开始的表格型 Q-Learning
4. **理解** 探索与利用的平衡策略
5. **对比** Q-Learning 和 SARSA 的行为差异
6. **应用** 高级技巧如 Double Q-Learning

## 前置知识

- 马尔可夫决策过程 (MDP) 基本概念
- Python 和 NumPy 基础
- 概率论基础

## 预计时间

90-120 分钟（可分多次完成）

---

# 第一部分：理论基础

## 1.1 从动态规划到无模型学习

### 动态规划的局限性

在 MDP 基础模块中，我们学习了动态规划 (DP) 方法求解最优策略。DP 需要：

1. **完整的环境模型**：状态转移概率 $P(s'|s,a)$ 和奖励函数 $R(s,a,s')$
2. **遍历所有状态和动作**：计算复杂度随状态空间增大而爆炸

**现实问题**：
- 转移概率通常未知（如何精确计算开车时每个操作的后果？）
- 状态空间可能极其庞大（围棋约有 $10^{170}$ 种状态）

### 无模型强化学习

**核心思想**：通过与环境**交互采样**学习最优策略，无需事先知道环境模型。

```
┌─────────────────────────────────────────────────────────────┐
│                    强化学习方法分类                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  ┌─────────────┐           ┌─────────────────────────────┐  │
│  │  基于模型   │           │        无模型 (Model-Free)   │  │
│  │ (Model-Based)│          ├──────────────┬──────────────┤  │
│  │             │           │  基于价值    │  基于策略     │  │
│  │ • 动态规划   │           │ (Value-Based)│(Policy-Based)│  │
│  │ • 模型预测控制│          │              │              │  │
│  │ • Dyna-Q    │           │ • Q-Learning │ • REINFORCE  │  │
│  └─────────────┘           │ • SARSA      │ • Actor-Critic│  │
│                            │ • DQN        │ • PPO        │  │
│                            └──────────────┴──────────────┘  │
└─────────────────────────────────────────────────────────────┘
```

## 1.2 时序差分学习 (TD Learning)

### 蒙特卡洛 vs 时序差分

**蒙特卡洛方法** (Monte Carlo, MC)：

$$V(S_t) \leftarrow V(S_t) + \alpha \left[ G_t - V(S_t) \right]$$

- 需要等待**回合结束**才能更新
- $G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + ...$ 是完整回合的实际累积回报
- 无偏估计，但方差较大

**时序差分方法** (Temporal Difference, TD)：

$$V(S_t) \leftarrow V(S_t) + \alpha \left[ R_{t+1} + \gamma V(S_{t+1}) - V(S_t) \right]$$

- **每一步**都可以更新
- 使用**自举** (Bootstrapping)：用估计值更新估计值
- 有偏估计，但方差较小

### TD 误差

**TD 目标** (TD Target):

$$\text{TD Target} = R_{t+1} + \gamma V(S_{t+1})$$

**TD 误差** (TD Error):

$$\delta_t = R_{t+1} + \gamma V(S_{t+1}) - V(S_t)$$

**直觉理解**：
- $\delta_t > 0$：实际比预期好，应增大 $V(S_t)$
- $\delta_t < 0$：实际比预期差，应减小 $V(S_t)$
- $\delta_t = 0$：预测准确，价值已收敛

## 1.3 Q-Learning 算法

### 核心公式

Q-Learning 是一种**离策略** (Off-Policy) TD 控制算法，直接学习最优动作价值函数 $Q^*$：

$$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_{a} Q(S_{t+1}, a) - Q(S_t, A_t) \right]$$

**分解来看**：

```
Q(S_t, A_t) ← Q(S_t, A_t) + α × [R_{t+1} + γ max_a Q(S_{t+1}, a) - Q(S_t, A_t)]
              \_________/       \___________________________________/   \________/
               旧估计                      TD 目标                       旧估计
                                 \________________________________________________/
                                                    TD 误差 δ_t
```

### 关键特性

1. **离策略 (Off-Policy)**：学习最优策略，不受探索策略影响
2. **使用 max 操作**：假设未来总是采取最优动作
3. **收敛性保证**：在一定条件下收敛到 $Q^*$

### 算法伪代码

```
算法: Q-Learning

输入: 状态空间 S, 动作空间 A, 学习率 α, 折扣因子 γ, 探索率 ε
输出: 最优 Q 函数

1. 初始化 Q(s, a) = 0，对于所有 s ∈ S, a ∈ A
2. 对于每个回合:
   a. 初始化状态 S
   b. 重复 (对于回合中的每一步):
      i.   使用 ε-greedy 从 Q 选择动作 A
      ii.  执行动作 A，观察奖励 R 和下一状态 S'
      iii. Q(S, A) ← Q(S, A) + α[R + γ max_a Q(S', a) - Q(S, A)]
      iv.  S ← S'
   c. 直到 S 是终止状态
3. 返回 Q
```

---

# 第二部分：代码实现

## 2.1 环境配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

import numpy as np
from collections import defaultdict
from typing import Tuple, List, Dict, Any, Optional
from dataclasses import dataclass, field
import matplotlib.pyplot as plt

# ============================================================
# 配置参数
# ============================================================

# 设置随机种子，确保结果可重复
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 可视化配置
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("环境配置完成！")
print(f"NumPy 版本: {np.__version__}")

## 2.2 实现悬崖行走环境

悬崖行走 (Cliff Walking) 是经典的强化学习测试环境，用于对比不同算法的行为特性：

```
┌─────────────────────────────────────────────┐
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 0
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 1
│ .  .  .  .  .  .  .  .  .  .  .  .  │  row 2
│ S  C  C  C  C  C  C  C  C  C  C  G  │  row 3
└─────────────────────────────────────────────┘
  0  1  2  3  4  5  6  7  8  9 10 11

S: 起点    G: 目标    C: 悬崖
```

**规则**：
- 动作空间：上(0)、右(1)、下(2)、左(3)
- 每步奖励：-1（鼓励快速到达）
- 掉入悬崖：-100 奖励，重置到起点

In [ ]:
class CliffWalkingEnv:
    """
    悬崖行走环境
    
    核心思想：
        经典的强化学习测试环境，用于演示 Q-Learning 和 SARSA 的行为差异。
        智能体需要从起点 S 到达目标 G，同时避免掉入悬崖 C。
    
    数学原理：
        这是一个确定性 MDP，状态转移概率 P(s'|s,a) ∈ {0, 1}。
        奖励函数设计鼓励快速到达目标并惩罚危险行为。
    
    问题背景：
        该环境用于展示离策略 (Q-Learning) 和在策略 (SARSA) 算法的关键差异：
        - Q-Learning 学习最优但风险高的路径（沿悬崖边）
        - SARSA 学习安全但较长的路径（远离悬崖）
    """
    
    # 动作定义：(行偏移, 列偏移)
    ACTIONS = {
        0: (-1, 0),   # 上
        1: (0, 1),    # 右
        2: (1, 0),    # 下
        3: (0, -1)    # 左
    }
    ACTION_NAMES = ['上', '右', '下', '左']
    
    def __init__(self, height: int = 4, width: int = 12):
        """
        初始化环境
        
        Args:
            height: 网格高度，默认 4
            width: 网格宽度，默认 12
        """
        self.height = height
        self.width = width
        
        # 特殊位置定义
        self.start = (height - 1, 0)           # 起点：左下角
        self.goal = (height - 1, width - 1)    # 终点：右下角
        self.cliff = [(height - 1, j) for j in range(1, width - 1)]  # 悬崖：底部中间
        
        # 当前状态
        self.state = self.start
        self.n_actions = 4
        
    def reset(self) -> Tuple[int, int]:
        """重置环境到初始状态"""
        self.state = self.start
        return self.state
    
    def step(self, action: int) -> Tuple[Tuple[int, int], float, bool]:
        """
        执行动作
        
        Args:
            action: 动作索引 (0-3)
            
        Returns:
            (next_state, reward, done) 三元组
        """
        # 计算下一位置（边界裁剪，撞墙则留在原地）
        di, dj = self.ACTIONS[action]
        new_i = np.clip(self.state[0] + di, 0, self.height - 1)
        new_j = np.clip(self.state[1] + dj, 0, self.width - 1)
        next_state = (int(new_i), int(new_j))
        
        # 检查是否掉入悬崖
        if next_state in self.cliff:
            self.state = self.start  # 重置到起点
            return self.state, -100.0, False
        
        self.state = next_state
        
        # 检查是否到达目标
        if self.state == self.goal:
            return self.state, 0.0, True
        
        return self.state, -1.0, False
    
    def render(self, path: Optional[List[Tuple[int, int]]] = None) -> None:
        """可视化环境"""
        grid = [['.' for _ in range(self.width)] for _ in range(self.height)]
        
        # 标记悬崖
        for pos in self.cliff:
            grid[pos[0]][pos[1]] = 'C'
        
        # 标记起点和终点
        grid[self.start[0]][self.start[1]] = 'S'
        grid[self.goal[0]][self.goal[1]] = 'G'
        
        # 标记路径
        if path:
            for pos in path[1:-1]:
                if pos not in self.cliff and pos != self.start and pos != self.goal:
                    grid[pos[0]][pos[1]] = '*'
        
        # 打印网格
        print("┌" + "─" * (self.width * 2 + 1) + "┐")
        for row in grid:
            print("│ " + " ".join(row) + " │")
        print("└" + "─" * (self.width * 2 + 1) + "┘")


# 测试环境
env = CliffWalkingEnv()
print("悬崖行走环境:")
env.render()
print(f"\n起点: {env.start}")
print(f"终点: {env.goal}")
print(f"悬崖位置: {env.cliff[:3]}...{env.cliff[-1]}")
print(f"状态空间: {env.height * env.width} 个状态")
print(f"动作空间: {env.n_actions} 个动作")

### 交互式探索：手动控制智能体

让我们手动执行几个动作，感受环境的奖励设计：

In [ ]:
# 手动探索环境
env = CliffWalkingEnv()
state = env.reset()

print("=" * 50)
print("手动探索环境")
print("=" * 50)

# 执行一系列动作
actions = [0, 0, 1, 1, 1]  # 上、上、右、右、右
action_names = env.ACTION_NAMES

print(f"\n初始状态: {state}")
total_reward = 0

for action in actions:
    next_state, reward, done = env.step(action)
    total_reward += reward
    print(f"动作: {action_names[action]:2s} | 状态: {state} -> {next_state} | 奖励: {reward:+.0f} | 累计: {total_reward:+.0f}")
    state = next_state
    if done:
        print("到达目标！")
        break

print(f"\n最终累计奖励: {total_reward}")

# 演示掉入悬崖
print("\n" + "=" * 50)
print("演示掉入悬崖")
print("=" * 50)

env.reset()
next_state, reward, done = env.step(1)  # 从起点向右 -> 掉入悬崖
print(f"从起点向右: 奖励 = {reward}, 状态重置到 {next_state}")

## 2.3 实现 Q-Learning 智能体

现在让我们从零实现一个 Q-Learning 智能体：

In [ ]:
@dataclass
class TrainingMetrics:
    """
    训练指标记录
    
    用于监控学习进度和分析算法性能。
    """
    episode_rewards: List[float] = field(default_factory=list)
    episode_lengths: List[int] = field(default_factory=list)
    epsilon_history: List[float] = field(default_factory=list)
    
    def get_moving_average(self, window: int = 10) -> np.ndarray:
        """计算移动平均"""
        if len(self.episode_rewards) < window:
            return np.array(self.episode_rewards)
        return np.convolve(
            self.episode_rewards,
            np.ones(window) / window,
            mode='valid'
        )


class QLearningAgent:
    """
    表格型 Q-Learning 智能体
    
    核心思想：
        Q-Learning 是一种离策略 (off-policy) TD 控制算法，
        通过与环境交互直接学习最优动作价值函数 Q*。
    
    数学原理：
        更新公式：
        Q(S_t, A_t) ← Q(S_t, A_t) + α [R_{t+1} + γ max_a Q(S_{t+1}, a) - Q(S_t, A_t)]
        
        其中：
        - α: 学习率，控制更新步长
        - γ: 折扣因子，权衡即时与未来奖励
        - max_a Q(S_{t+1}, a): 下一状态的最优动作价值（离策略的关键）
    
    算法对比：
        vs SARSA：Q-Learning 使用 max（离策略），SARSA 使用实际动作（在策略）
        vs MC：Q-Learning 每步更新（TD），MC 需要完整回合
    
    复杂度：
        - 空间：O(|S| × |A|)，存储 Q 表
        - 时间：每步 O(|A|)，计算 max
    
    Attributes:
        q_table: Q 值表，字典形式 {state: [Q(s,a0), Q(s,a1), ...]}
        lr: 学习率 α
        gamma: 折扣因子 γ
        epsilon: 探索率
    """
    
    def __init__(
        self,
        n_actions: int,
        learning_rate: float = 0.1,
        discount_factor: float = 0.99,
        epsilon: float = 1.0,
        epsilon_decay: float = 0.995,
        epsilon_min: float = 0.01
    ):
        """
        初始化 Q-Learning 智能体
        
        Args:
            n_actions: 动作空间大小
            learning_rate: 学习率，控制 Q 值更新步长
            discount_factor: 折扣因子，权衡即时与未来奖励
            epsilon: 初始探索率
            epsilon_decay: 探索率衰减系数
            epsilon_min: 最小探索率
        """
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # Q 表：使用 defaultdict 自动初始化未访问状态
        self.q_table: Dict[Any, np.ndarray] = defaultdict(
            lambda: np.zeros(n_actions)
        )
        
        # 训练指标
        self.metrics = TrainingMetrics()
        
    def get_action(self, state: Any, training: bool = True) -> int:
        """
        使用 ε-greedy 策略选择动作
        
        ε-greedy 策略：
            - 以概率 ε 随机选择（探索）
            - 以概率 1-ε 选择 Q 值最大的动作（利用）
        
        Args:
            state: 当前状态
            training: 是否处于训练模式
            
        Returns:
            选择的动作索引
        """
        # 训练时以 ε 概率随机探索
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        
        # 利用：选择 Q 值最大的动作
        # 当存在多个最大值时，随机选择（打破平局）
        q_values = self.q_table[state]
        max_q = np.max(q_values)
        max_actions = np.where(np.isclose(q_values, max_q))[0]
        return np.random.choice(max_actions)
    
    def update(
        self,
        state: Any,
        action: int,
        reward: float,
        next_state: Any,
        done: bool
    ) -> float:
        """
        Q-Learning 更新规则
        
        更新公式：
            Q(S,A) ← Q(S,A) + α[R + γ max_a Q(S',a) - Q(S,A)]
        
        Args:
            state: 当前状态
            action: 执行的动作
            reward: 获得的奖励
            next_state: 下一状态
            done: 是否终止
            
        Returns:
            TD 误差
        """
        current_q = self.q_table[state][action]
        
        # 计算 TD 目标
        if done:
            target = reward  # 终止状态没有后续奖励
        else:
            # Q-Learning 核心：使用 max 选择下一状态的最优动作
            target = reward + self.gamma * np.max(self.q_table[next_state])
        
        # TD 误差：反映预测与实际的差距
        td_error = target - current_q
        
        # 更新 Q 值：沿着减小 TD 误差的方向调整
        self.q_table[state][action] += self.lr * td_error
        
        return td_error
    
    def decay_epsilon(self) -> None:
        """衰减探索率"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


# 创建智能体实例
agent = QLearningAgent(
    n_actions=4,
    learning_rate=0.5,
    discount_factor=0.99,
    epsilon=0.1,
    epsilon_decay=1.0,  # 保持固定探索率
    epsilon_min=0.1
)

print("Q-Learning 智能体配置:")
print(f"  动作数量: {agent.n_actions}")
print(f"  学习率 α: {agent.lr}")
print(f"  折扣因子 γ: {agent.gamma}")
print(f"  探索率 ε: {agent.epsilon}")

### 交互式探索：观察 Q 值更新

让我们手动执行几步，观察 Q 值如何更新：

In [ ]:
# 观察 Q 值更新过程
env = CliffWalkingEnv()
agent = QLearningAgent(n_actions=4, learning_rate=0.5, discount_factor=0.9)

print("=" * 60)
print("观察 Q 值更新过程")
print("=" * 60)

state = env.reset()

# 执行几步并观察 Q 值变化
actions_to_take = [0, 0, 1, 1, 2]  # 上、上、右、右、下

for i, action in enumerate(actions_to_take):
    print(f"\n步骤 {i+1}:")
    print(f"  当前状态: {state}")
    print(f"  更新前 Q 值: {agent.q_table[state]}")
    
    next_state, reward, done = env.step(action)
    td_error = agent.update(state, action, reward, next_state, done)
    
    print(f"  执行动作: {env.ACTION_NAMES[action]}")
    print(f"  获得奖励: {reward}")
    print(f"  TD 误差: {td_error:.4f}")
    print(f"  更新后 Q 值: {agent.q_table[state]}")
    
    state = next_state
    if done:
        print("  到达目标！")
        break

## 2.4 训练循环实现

In [ ]:
def train_q_learning(
    env: CliffWalkingEnv,
    agent: QLearningAgent,
    episodes: int = 500,
    max_steps: int = 200,
    verbose: bool = True,
    log_interval: int = 100
) -> TrainingMetrics:
    """
    训练 Q-Learning 智能体
    
    Args:
        env: 环境实例
        agent: Q-Learning 智能体
        episodes: 训练回合数
        max_steps: 每回合最大步数
        verbose: 是否打印训练进度
        log_interval: 日志打印间隔
        
    Returns:
        训练历史记录
    """
    metrics = TrainingMetrics()
    
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0.0
        steps = 0
        
        for step in range(max_steps):
            # 选择动作
            action = agent.get_action(state, training=True)
            
            # 执行动作
            next_state, reward, done = env.step(action)
            
            # 更新 Q 值
            agent.update(state, action, reward, next_state, done)
            
            state = next_state
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        # 衰减探索率
        agent.decay_epsilon()
        
        # 记录历史
        metrics.episode_rewards.append(total_reward)
        metrics.episode_lengths.append(steps)
        metrics.epsilon_history.append(agent.epsilon)
        
        # 打印进度
        if verbose and (episode + 1) % log_interval == 0:
            avg_reward = np.mean(metrics.episode_rewards[-log_interval:])
            avg_steps = np.mean(metrics.episode_lengths[-log_interval:])
            print(f"Episode {episode + 1:4d} | "
                  f"Avg Reward: {avg_reward:8.2f} | "
                  f"Avg Steps: {avg_steps:6.1f} | "
                  f"ε: {agent.epsilon:.4f}")
    
    agent.metrics = metrics
    return metrics


# 训练智能体
print("开始训练 Q-Learning...\n")
env = CliffWalkingEnv()
agent = QLearningAgent(
    n_actions=4,
    learning_rate=0.5,
    discount_factor=0.99,
    epsilon=0.1,
    epsilon_decay=1.0,  # 不衰减，便于观察行为
    epsilon_min=0.1
)

metrics = train_q_learning(env, agent, episodes=500)

print(f"\n训练完成！")
print(f"最后 100 回合平均奖励: {np.mean(metrics.episode_rewards[-100:]):.2f}")
print(f"Q 表大小: {len(agent.q_table)} 个状态")

## 2.5 可视化学习过程

In [ ]:
def plot_training_curves(metrics: TrainingMetrics, window: int = 10, title: str = "Q-Learning"):
    """绘制训练曲线"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 奖励曲线
    rewards = metrics.episode_rewards
    smoothed_rewards = np.convolve(
        rewards, np.ones(window)/window, mode='valid'
    )
    
    axes[0].plot(rewards, alpha=0.3, color='blue', label='原始奖励')
    axes[0].plot(range(window-1, len(rewards)), smoothed_rewards, 
                 color='blue', linewidth=2, label=f'{window} 回合移动平均')
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Total Reward')
    axes[0].set_title(f'{title} 学习曲线')
    axes[0].legend()
    axes[0].axhline(y=-13, color='green', linestyle='--', alpha=0.7, label='最优 (理论)')
    
    # 步数曲线
    steps = metrics.episode_lengths
    smoothed_steps = np.convolve(
        steps, np.ones(window)/window, mode='valid'
    )
    
    axes[1].plot(steps, alpha=0.3, color='green', label='原始步数')
    axes[1].plot(range(window-1, len(steps)), smoothed_steps,
                 color='green', linewidth=2, label=f'{window} 回合移动平均')
    axes[1].set_xlabel('Episode')
    axes[1].set_ylabel('Steps')
    axes[1].set_title('每回合步数')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()


plot_training_curves(metrics)

## 2.6 提取并可视化学到的策略

In [ ]:
def extract_path(agent: QLearningAgent, env: CliffWalkingEnv, max_steps: int = 50) -> List[Tuple[int, int]]:
    """从训练好的智能体提取贪心策略路径"""
    state = env.reset()
    path = [state]
    
    for _ in range(max_steps):
        action = agent.get_action(state, training=False)  # 不探索
        next_state, _, done = env.step(action)
        path.append(next_state)
        state = next_state
        if done:
            break
    
    return path


def visualize_policy(agent: QLearningAgent, env: CliffWalkingEnv):
    """可视化学到的策略"""
    arrow_map = {0: '↑', 1: '→', 2: '↓', 3: '←'}
    
    print("学到的策略 (贪心):")
    print("┌" + "───" * env.width + "┐")
    
    for i in range(env.height):
        row = "│"
        for j in range(env.width):
            state = (i, j)
            if state == env.start:
                row += " S "
            elif state == env.goal:
                row += " G "
            elif state in env.cliff:
                row += " C "
            elif state in agent.q_table:
                best_action = np.argmax(agent.q_table[state])
                row += f" {arrow_map[best_action]} "
            else:
                row += " . "
        print(row + "│")
    
    print("└" + "───" * env.width + "┘")


# 可视化策略
print("=" * 60)
print("Q-Learning 学到的策略")
print("=" * 60)
visualize_policy(agent, env)

# 提取并显示路径
print("\n学到的路径:")
path = extract_path(agent, env)
env.reset()
env.render(path)
print(f"路径长度: {len(path) - 1} 步")

## 2.7 Q 值表可视化

In [ ]:
def visualize_q_table(agent: QLearningAgent, env: CliffWalkingEnv):
    """可视化 Q 表和价值函数"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 准备价值函数数据
    v_table = np.zeros((env.height, env.width))
    for i in range(env.height):
        for j in range(env.width):
            state = (i, j)
            if state in agent.q_table:
                v_table[i, j] = np.max(agent.q_table[state])
            else:
                v_table[i, j] = 0
    
    # 价值函数热力图
    im = axes[0].imshow(v_table, cmap='RdYlGn', aspect='auto')
    axes[0].set_title('状态价值函数 V(s) = max_a Q(s,a)')
    axes[0].set_xlabel('列')
    axes[0].set_ylabel('行')
    plt.colorbar(im, ax=axes[0])
    
    # 标记悬崖位置
    for pos in env.cliff:
        axes[0].add_patch(plt.Rectangle(
            (pos[1]-0.5, pos[0]-0.5), 1, 1,
            fill=True, color='black', alpha=0.5
        ))
    
    # Q 值分布直方图
    q_values = []
    for state, q_array in agent.q_table.items():
        if state not in env.cliff:
            q_values.extend(q_array.tolist())
    
    axes[1].hist(q_values, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[1].set_xlabel('Q Value')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Q 值分布')
    
    plt.tight_layout()
    plt.show()


visualize_q_table(agent, env)

---

# 第三部分：Q-Learning vs SARSA 对比

## 3.1 SARSA 智能体实现

In [ ]:
class SARSAAgent:
    """
    SARSA 智能体 (On-Policy TD Control)
    
    核心思想：
        SARSA (State-Action-Reward-State-Action) 是一种在策略 (on-policy) 
        TD 控制算法，学习当前行为策略的价值函数 Q^π。
    
    数学原理：
        更新公式：
        Q(S_t, A_t) ← Q(S_t, A_t) + α [R_{t+1} + γ Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t)]
        
        与 Q-Learning 的关键区别：
        - 使用实际采取的下一动作 A_{t+1}，而非 max
        - 学习的是当前 ε-greedy 策略的价值，而非最优策略
    
    算法对比：
        在悬崖行走环境中：
        - Q-Learning：学习沿悬崖边的最短路径（因为更新不考虑探索）
        - SARSA：学习远离悬崖的安全路径（考虑探索时掉下悬崖的可能）
    """
    
    def __init__(
        self,
        n_actions: int,
        learning_rate: float = 0.1,
        discount_factor: float = 0.99,
        epsilon: float = 1.0,
        epsilon_decay: float = 0.995,
        epsilon_min: float = 0.01
    ):
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.q_table: Dict[Any, np.ndarray] = defaultdict(
            lambda: np.zeros(n_actions)
        )
        self.metrics = TrainingMetrics()
        
    def get_action(self, state: Any, training: bool = True) -> int:
        """ε-greedy 策略选择动作"""
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        q_values = self.q_table[state]
        max_q = np.max(q_values)
        max_actions = np.where(np.isclose(q_values, max_q))[0]
        return np.random.choice(max_actions)
    
    def update(
        self,
        state: Any,
        action: int,
        reward: float,
        next_state: Any,
        next_action: int,  # SARSA 需要下一个动作！
        done: bool
    ) -> float:
        """
        SARSA 更新规则
        
        与 Q-Learning 的关键区别：使用实际的 next_action，而非 max
        """
        current_q = self.q_table[state][action]
        
        if done:
            target = reward
        else:
            # SARSA 核心：使用实际的 next_action
            target = reward + self.gamma * self.q_table[next_state][next_action]
        
        td_error = target - current_q
        self.q_table[state][action] += self.lr * td_error
        
        return td_error
    
    def decay_epsilon(self) -> None:
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def train_sarsa(
    env: CliffWalkingEnv,
    agent: SARSAAgent,
    episodes: int = 500,
    max_steps: int = 200,
    verbose: bool = True,
    log_interval: int = 100
) -> TrainingMetrics:
    """训练 SARSA 智能体"""
    metrics = TrainingMetrics()
    
    for episode in range(episodes):
        state = env.reset()
        # SARSA: 先选择初始动作
        action = agent.get_action(state, training=True)
        
        total_reward = 0.0
        steps = 0
        
        for _ in range(max_steps):
            # 执行动作
            next_state, reward, done = env.step(action)
            
            # SARSA: 在更新前选择下一个动作
            next_action = agent.get_action(next_state, training=True)
            
            # SARSA 更新需要 next_action
            agent.update(state, action, reward, next_state, next_action, done)
            
            # 状态和动作传递
            state = next_state
            action = next_action  # 关键！
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        agent.decay_epsilon()
        
        metrics.episode_rewards.append(total_reward)
        metrics.episode_lengths.append(steps)
        metrics.epsilon_history.append(agent.epsilon)
        
        if verbose and (episode + 1) % log_interval == 0:
            avg_reward = np.mean(metrics.episode_rewards[-log_interval:])
            avg_steps = np.mean(metrics.episode_lengths[-log_interval:])
            print(f"Episode {episode + 1:4d} | "
                  f"Avg Reward: {avg_reward:8.2f} | "
                  f"Avg Steps: {avg_steps:6.1f} | "
                  f"ε: {agent.epsilon:.4f}")
    
    agent.metrics = metrics
    return metrics


print("SARSA 智能体和训练函数定义完成！")

## 3.2 对比实验

In [ ]:
# ============================================================
# 对比实验：Q-Learning vs SARSA
# ============================================================

print("=" * 60)
print("悬崖行走环境: Q-Learning vs SARSA 对比实验")
print("=" * 60)

# 实验参数
EPISODES = 500
LEARNING_RATE = 0.5
EPSILON = 0.1  # 固定探索率，便于观察行为差异

# 创建环境
env = CliffWalkingEnv()

# Q-Learning 智能体
q_agent = QLearningAgent(
    n_actions=4,
    learning_rate=LEARNING_RATE,
    epsilon=EPSILON,
    epsilon_decay=1.0,  # 不衰减
    epsilon_min=EPSILON
)

# SARSA 智能体
sarsa_agent = SARSAAgent(
    n_actions=4,
    learning_rate=LEARNING_RATE,
    epsilon=EPSILON,
    epsilon_decay=1.0,
    epsilon_min=EPSILON
)

# 训练
print("\n训练 Q-Learning...")
q_metrics = train_q_learning(env, q_agent, episodes=EPISODES, verbose=True)

print("\n训练 SARSA...")
sarsa_metrics = train_sarsa(env, sarsa_agent, episodes=EPISODES, verbose=True)

print("\n训练完成！")

## 3.3 可视化对比结果

In [ ]:
def plot_comparison(q_metrics: TrainingMetrics, sarsa_metrics: TrainingMetrics, window: int = 10):
    """绘制 Q-Learning 和 SARSA 的学习曲线对比"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 奖励曲线
    q_smooth = np.convolve(q_metrics.episode_rewards, np.ones(window)/window, mode='valid')
    sarsa_smooth = np.convolve(sarsa_metrics.episode_rewards, np.ones(window)/window, mode='valid')
    
    axes[0].plot(q_smooth, label='Q-Learning', color='blue', alpha=0.8, linewidth=2)
    axes[0].plot(sarsa_smooth, label='SARSA', color='red', alpha=0.8, linewidth=2)
    axes[0].axhline(y=-13, color='green', linestyle='--', alpha=0.7, label='最优路径 (-13)')
    axes[0].set_xlabel('Episode')
    axes[0].set_ylabel('Total Reward')
    axes[0].set_title('学习曲线: 回合奖励')
    axes[0].legend()
    
    # 步数曲线
    q_steps_smooth = np.convolve(q_metrics.episode_lengths, np.ones(window)/window, mode='valid')
    sarsa_steps_smooth = np.convolve(sarsa_metrics.episode_lengths, np.ones(window)/window, mode='valid')
    
    axes[1].plot(q_steps_smooth, label='Q-Learning', color='blue', alpha=0.8, linewidth=2)
    axes[1].plot(sarsa_steps_smooth, label='SARSA', color='red', alpha=0.8, linewidth=2)
    axes[1].set_xlabel('Episode')
    axes[1].set_ylabel('Steps')
    axes[1].set_title('学习曲线: 回合步数')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # 统计
    print("\n" + "=" * 50)
    print("最后 100 回合统计")
    print("=" * 50)
    print(f"Q-Learning: 平均奖励 = {np.mean(q_metrics.episode_rewards[-100:]):.2f}")
    print(f"SARSA:      平均奖励 = {np.mean(sarsa_metrics.episode_rewards[-100:]):.2f}")


plot_comparison(q_metrics, sarsa_metrics)

## 3.4 提取并对比学到的路径

In [ ]:
# 提取路径
print("\n" + "=" * 60)
print("学到的策略路径对比")
print("=" * 60)

print("\nQ-Learning 学到的路径 (倾向最短路径，沿悬崖边):")
q_path = extract_path(q_agent, env)
env.reset()
env.render(q_path)
print(f"路径长度: {len(q_path) - 1} 步")

print("\nSARSA 学到的路径 (倾向安全路径，远离悬崖):")

# SARSA 需要稍微修改 extract_path
def extract_path_sarsa(agent: SARSAAgent, env: CliffWalkingEnv, max_steps: int = 50):
    state = env.reset()
    path = [state]
    for _ in range(max_steps):
        action = agent.get_action(state, training=False)
        next_state, _, done = env.step(action)
        path.append(next_state)
        state = next_state
        if done:
            break
    return path

sarsa_path = extract_path_sarsa(sarsa_agent, env)
env.reset()
env.render(sarsa_path)
print(f"路径长度: {len(sarsa_path) - 1} 步")

## 3.5 行为差异分析

### 为什么 Q-Learning 选择悬崖边路径？

Q-Learning 更新使用 `max`，学习的是**最优策略的价值**：
- 假设执行最优策略，不会掉入悬崖
- 沿悬崖边的路径最短，奖励最高
- 但训练时的 ε-greedy 探索会导致实际掉入悬崖

**结果**：学到的策略是最优的，但训练过程中经常失败

### 为什么 SARSA 选择安全路径？

SARSA 使用实际采取的动作，学习的是**当前 ε-greedy 策略的价值**：
- 考虑到探索时可能随机选择动作
- 靠近悬崖时，探索可能导致掉落
- 因此远离悬崖的路径价值更高

**结果**：学到的策略更保守，但训练过程更稳定

In [ ]:
# 可视化价值函数对比
def visualize_value_comparison(q_agent, sarsa_agent, env):
    """对比两种算法学到的价值函数"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for idx, (agent, name) in enumerate([(q_agent, 'Q-Learning'), (sarsa_agent, 'SARSA')]):
        v_table = np.zeros((env.height, env.width))
        for i in range(env.height):
            for j in range(env.width):
                state = (i, j)
                if state in agent.q_table:
                    v_table[i, j] = np.max(agent.q_table[state])
        
        im = axes[idx].imshow(v_table, cmap='RdYlGn', aspect='auto')
        axes[idx].set_title(f'{name} 价值函数 V(s)')
        axes[idx].set_xlabel('列')
        axes[idx].set_ylabel('行')
        plt.colorbar(im, ax=axes[idx])
        
        # 标记悬崖
        for pos in env.cliff:
            axes[idx].add_patch(plt.Rectangle(
                (pos[1]-0.5, pos[0]-0.5), 1, 1,
                fill=True, color='black', alpha=0.5
            ))
    
    plt.tight_layout()
    plt.show()


visualize_value_comparison(q_agent, sarsa_agent, env)

---

# 第四部分：高级技巧

## 4.1 Double Q-Learning

### 过估计问题

Q-Learning 的 `max` 操作导致系统性过估计：

- 假设 Q 值估计有噪声：$\hat{Q}(s,a) = Q^*(s,a) + \epsilon_a$
- $\mathbb{E}[\max_a \hat{Q}(s,a)] \geq \max_a \mathbb{E}[\hat{Q}(s,a)]$
- 这种正偏差会通过 bootstrapping 累积

### 解决方案：Double Q-Learning

解耦动作选择和价值评估：

$$Q_1(S,A) \leftarrow Q_1(S,A) + \alpha[R + \gamma Q_2(S', \arg\max_a Q_1(S',a)) - Q_1(S,A)]$$

In [ ]:
class DoubleQLearningAgent:
    """
    Double Q-Learning 智能体
    
    核心思想：
        通过维护两个 Q 表，解耦动作选择和价值评估，减少过估计偏差。
    
    数学原理：
        以 0.5 概率选择更新 Q1 或 Q2：
        - 更新 Q1：用 Q1 选择动作，Q2 评估价值
        - 更新 Q2：用 Q2 选择动作，Q1 评估价值
    """
    
    def __init__(
        self,
        n_actions: int,
        learning_rate: float = 0.1,
        discount_factor: float = 0.99,
        epsilon: float = 1.0,
        epsilon_decay: float = 0.995,
        epsilon_min: float = 0.01
    ):
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # 两个独立的 Q 表
        self.q_table1: Dict[Any, np.ndarray] = defaultdict(
            lambda: np.zeros(n_actions)
        )
        self.q_table2: Dict[Any, np.ndarray] = defaultdict(
            lambda: np.zeros(n_actions)
        )
        
        self.metrics = TrainingMetrics()
        
    def get_action(self, state: Any, training: bool = True) -> int:
        """使用两个 Q 表的和选择动作"""
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.n_actions)
        
        combined_q = self.q_table1[state] + self.q_table2[state]
        max_q = np.max(combined_q)
        max_actions = np.where(np.isclose(combined_q, max_q))[0]
        return np.random.choice(max_actions)
    
    def update(
        self,
        state: Any,
        action: int,
        reward: float,
        next_state: Any,
        done: bool
    ) -> float:
        """Double Q-Learning 更新"""
        if np.random.random() < 0.5:
            # 更新 Q1：用 Q1 选择动作，Q2 评估
            current_q = self.q_table1[state][action]
            if done:
                target = reward
            else:
                best_action = np.argmax(self.q_table1[next_state])
                target = reward + self.gamma * self.q_table2[next_state][best_action]
            td_error = target - current_q
            self.q_table1[state][action] += self.lr * td_error
        else:
            # 更新 Q2：用 Q2 选择动作，Q1 评估
            current_q = self.q_table2[state][action]
            if done:
                target = reward
            else:
                best_action = np.argmax(self.q_table2[next_state])
                target = reward + self.gamma * self.q_table1[next_state][best_action]
            td_error = target - current_q
            self.q_table2[state][action] += self.lr * td_error
        
        return td_error
    
    def decay_epsilon(self) -> None:
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
    # 兼容属性
    @property
    def q_table(self):
        """返回合并的 Q 表"""
        combined = defaultdict(lambda: np.zeros(self.n_actions))
        for state in set(self.q_table1.keys()) | set(self.q_table2.keys()):
            combined[state] = (self.q_table1[state] + self.q_table2[state]) / 2
        return combined


print("Double Q-Learning 智能体定义完成！")

## 4.2 三种算法对比

In [ ]:
print("=" * 60)
print("Q-Learning vs Double Q-Learning vs SARSA 对比")
print("=" * 60)

env = CliffWalkingEnv()

# 创建三种智能体
q_agent = QLearningAgent(
    n_actions=4, learning_rate=0.5, epsilon=0.1,
    epsilon_decay=1.0, epsilon_min=0.1
)

double_q_agent = DoubleQLearningAgent(
    n_actions=4, learning_rate=0.5, epsilon=0.1,
    epsilon_decay=1.0, epsilon_min=0.1
)

sarsa_agent = SARSAAgent(
    n_actions=4, learning_rate=0.5, epsilon=0.1,
    epsilon_decay=1.0, epsilon_min=0.1
)

# 训练
print("\n训练 Q-Learning...")
q_metrics = train_q_learning(env, q_agent, episodes=500, verbose=False)
print(f"完成！最后100回合平均奖励: {np.mean(q_metrics.episode_rewards[-100:]):.2f}")

print("\n训练 Double Q-Learning...")
double_q_metrics = train_q_learning(env, double_q_agent, episodes=500, verbose=False)
print(f"完成！最后100回合平均奖励: {np.mean(double_q_metrics.episode_rewards[-100:]):.2f}")

print("\n训练 SARSA...")
sarsa_metrics = train_sarsa(env, sarsa_agent, episodes=500, verbose=False)
print(f"完成！最后100回合平均奖励: {np.mean(sarsa_metrics.episode_rewards[-100:]):.2f}")

In [ ]:
# 绘制三种算法对比
fig, ax = plt.subplots(figsize=(12, 6))

window = 10
q_smooth = np.convolve(q_metrics.episode_rewards, np.ones(window)/window, mode='valid')
double_smooth = np.convolve(double_q_metrics.episode_rewards, np.ones(window)/window, mode='valid')
sarsa_smooth = np.convolve(sarsa_metrics.episode_rewards, np.ones(window)/window, mode='valid')

ax.plot(q_smooth, label='Q-Learning', alpha=0.8, linewidth=2)
ax.plot(double_smooth, label='Double Q-Learning', alpha=0.8, linewidth=2)
ax.plot(sarsa_smooth, label='SARSA', alpha=0.8, linewidth=2)
ax.axhline(y=-13, color='green', linestyle='--', alpha=0.7, label='最优 (-13)')

ax.set_xlabel('Episode')
ax.set_ylabel('Total Reward')
ax.set_title('三种 TD 控制算法对比 (悬崖行走)')
ax.legend()
plt.tight_layout()
plt.show()

---

# 第五部分：总结与练习

## 5.1 核心要点回顾

### Q-Learning 算法

| 特性 | 描述 |
|------|------|
| 类型 | 离策略 (Off-Policy) TD 控制 |
| 更新公式 | $Q(S,A) \leftarrow Q(S,A) + \alpha[R + \gamma \max_a Q(S',a) - Q(S,A)]$ |
| 学习目标 | 最优动作价值函数 $Q^*$ |
| 优点 | 直接学习最优策略，样本可重用 |
| 缺点 | 可能过估计，探索时不安全 |

### 算法对比

| 算法 | 策略类型 | TD 目标 | 适用场景 |
|------|----------|---------|----------|
| Q-Learning | Off-Policy | $\max_a Q(S',a)$ | 追求最优性能 |
| SARSA | On-Policy | $Q(S',A')$ | 需要安全探索 |
| Double Q-Learning | Off-Policy | $Q_2(S', \arg\max_a Q_1(S',a))$ | 减少过估计 |

### 推荐超参数

| 参数 | 典型值 | 说明 |
|------|--------|------|
| $\alpha$ (学习率) | 0.1 ~ 0.5 | 表格型可用较大值 |
| $\gamma$ (折扣因子) | 0.99 | 接近 1 重视长期 |
| $\epsilon$ (初始探索率) | 1.0 | 从完全探索开始 |
| $\epsilon_{min}$ | 0.01 ~ 0.1 | 保持少量探索 |
| 衰减率 | 0.99 ~ 0.999 | 控制探索下降 |

## 5.2 单元测试

In [ ]:
def run_tests():
    """运行所有单元测试"""
    print("开始单元测试...\n")
    passed = 0
    failed = 0
    
    # 测试1: 环境基本功能
    try:
        env = CliffWalkingEnv()
        state = env.reset()
        assert state == (3, 0), f"起始状态错误: {state}"
        
        next_state, reward, done = env.step(0)  # 向上
        assert next_state == (2, 0), f"移动后状态错误: {next_state}"
        assert reward == -1.0, f"奖励错误: {reward}"
        
        print("测试1通过: 环境基本功能正常")
        passed += 1
    except AssertionError as e:
        print(f"测试1失败: {e}")
        failed += 1
    
    # 测试2: 悬崖惩罚
    try:
        env = CliffWalkingEnv()
        env.reset()
        next_state, reward, done = env.step(1)  # 向右进入悬崖
        assert reward == -100.0, f"悬崖惩罚错误: {reward}"
        assert next_state == env.start, "掉入悬崖后应重置到起点"
        
        print("测试2通过: 悬崖惩罚正确")
        passed += 1
    except AssertionError as e:
        print(f"测试2失败: {e}")
        failed += 1
    
    # 测试3: Q-Learning 更新
    try:
        agent = QLearningAgent(n_actions=4, learning_rate=0.5, discount_factor=0.9)
        state = (0, 0)
        next_state = (0, 1)
        
        assert agent.q_table[state][0] == 0.0, "初始 Q 值应为 0"
        
        td_error = agent.update(state, 0, -1.0, next_state, False)
        # Q(s,a) = 0 + 0.5 * (-1 + 0.9 * 0 - 0) = -0.5
        assert np.isclose(agent.q_table[state][0], -0.5), \
            f"Q值更新错误: {agent.q_table[state][0]}"
        
        print("测试3通过: Q-Learning 更新正确")
        passed += 1
    except AssertionError as e:
        print(f"测试3失败: {e}")
        failed += 1
    
    # 测试4: SARSA 更新
    try:
        agent = SARSAAgent(n_actions=4, learning_rate=0.5, discount_factor=0.9)
        state = (0, 0)
        next_state = (0, 1)
        
        agent.q_table[next_state] = np.array([1.0, 2.0, 0.0, 0.0])
        agent.update(state, 0, -1.0, next_state, 1, False)  # next_action=1
        # Q(s,a) = 0 + 0.5 * (-1 + 0.9 * 2.0 - 0) = 0.4
        assert np.isclose(agent.q_table[state][0], 0.4), \
            f"SARSA更新错误: {agent.q_table[state][0]}"
        
        print("测试4通过: SARSA 更新正确")
        passed += 1
    except AssertionError as e:
        print(f"测试4失败: {e}")
        failed += 1
    
    # 测试5: 训练收敛性
    try:
        env = CliffWalkingEnv()
        agent = QLearningAgent(
            n_actions=4, learning_rate=0.5,
            epsilon=0.1, epsilon_decay=1.0, epsilon_min=0.1
        )
        
        metrics = train_q_learning(env, agent, episodes=200, verbose=False)
        avg_reward = np.mean(metrics.episode_rewards[-50:])
        assert avg_reward > -100, f"训练未收敛: {avg_reward}"
        
        print(f"测试5通过: 训练收敛 (最后50回合平均奖励: {avg_reward:.2f})")
        passed += 1
    except AssertionError as e:
        print(f"测试5失败: {e}")
        failed += 1
    
    # 总结
    print(f"\n{'='*50}")
    print(f"测试完成: {passed} 通过, {failed} 失败")
    if failed == 0:
        print("所有测试通过！")
    print(f"{'='*50}")
    
    return failed == 0


run_tests()

## 5.3 练习题

### 练习1: 调整超参数

尝试修改学习率、折扣因子和探索率，观察对学习效果的影响。

### 练习2: 实现 Expected SARSA

Expected SARSA 使用期望而非采样：

$$Q(S,A) \leftarrow Q(S,A) + \alpha[R + \gamma \mathbb{E}_\pi[Q(S',A')] - Q(S,A)]$$

### 练习3: 在 Gymnasium 环境中测试

尝试在 Taxi-v3 或 FrozenLake-v1 环境中训练 Q-Learning 智能体。

---

## 参考资料

1. Watkins, C.J.C.H. (1989). *Learning from Delayed Rewards*. PhD Thesis.
2. Sutton, R.S. & Barto, A.G. (2018). *Reinforcement Learning: An Introduction*, 2nd ed. Chapter 6.
3. Van Hasselt, H. (2010). *Double Q-learning*. NeurIPS.

---

[返回目录](README.md)